In [1]:
import uuid
from text_process_tools import *  # 假设text_process_tools.py在同一目录下

In [ ]:
# 问题一：翻译一个非英语的句子
def test_1():
    sentence1 = Sentence(content=[uuid4()], language="fr", sentiment="neu")
    question = f"请你将句子{sentence1.id}翻译为英语"
    def eval_1(submit_sentence_id):
        obj = load_object_from_json(submit_sentence_id)
        if obj.language == "en" and obj.content==sentence1.content:
            return 1
        else:
            return 0
    return question, eval_1, [sentence1.id]

question, eval_fun, provided_info = test_1()
translated_id = translate_tool(provided_info[0], "en")
eval_fun(translated_id)


1

In [ ]:
# 问题二：将一篇文章从中文翻译为英文，法文，德文,不会超过最大长度限制的版本
Chaps = []
for i in range(3):
    Chap_sentences = []
    for i in range(3):
        s = Sentence(content=[uuid4()], language="zh")
        Chap_sentences.append(s)
    chap = Chapter(sentences=Chap_sentences, language="zh")
    Chaps.append(chap)
article = Article(chapters=Chaps, language="zh")

article_en_id = translate_tool(article.id, "en")

def test_2():
    Chaps = []
    for i in range(3):
        Chap_sentences = []
        for i in range(3):
            s = Sentence(content=[uuid4()], language="zh")
            Chap_sentences.append(s)
        chap = Chapter(sentences=Chap_sentences, language="zh")
        Chaps.append(chap)
    article = Article(chapters=Chaps, language="zh")
    question = f"请你将下面的文章翻译成英文，德文和法文"
    def eval_2(submit_article_ids):
        for aid in submit_article_ids:
            if 
        

In [19]:
# 问题三：将一篇文章从中文翻译为英文，法文，德文,会超过最大长度限制的版本
Chaps = []
for i in range(3):
    Chap_sentences = []
    for i in range(30):
        s = Sentence(content=[uuid4()], language="zh")
        Chap_sentences.append(s)
    chap = Chapter(sentences=Chap_sentences, language="zh")
    Chaps.append(chap)
article = Article(chapters=Chaps, language="zh")
print(article.word_count)

# 解决方案

chaps_id = split_article_tool(article.id)
print(chaps_id)
T_chaps_id = []
for chap_id in chaps_id:
    translated_chap_id = translate_tool(chap_id, "fr") # 涉及到三个语言
    T_chaps_id.append(translated_chap_id)
translated_article_id = construct_article_tool(chapters_id=T_chaps_id)
print(detect_language_tool(translated_article_id))
translated_article = load_object_from_json(translated_article_id)
print(set(translated_article.content)==set(article.content))

1346
[UUID('133ddec8-dd02-4b83-a5e3-93eb26af5432'), UUID('be9cca67-90c5-4ac7-aa3a-e5c7e7c1dd65'), UUID('121d0449-c89a-46a8-b51d-34f1a18fd7b2')]
fr
True


In [7]:
# 问题四：将所有情感为正向的句子拿出来组成一个篇章
Sentences_id = []
for i in range(10):
    s = Sentence(content=[uuid4()], language="zh")
    Sentences_id.append(s.id)

# 解决方案
Chap_id = []
for s_id in Sentences_id:
    trans_id = translate_tool(s_id, "en")
    senti = sentiment_analysis_sentence_tool(trans_id)
    print(senti)
    if senti=="pos":
        Chap_id.append(s_id)
chap_id = construct_chapter_tool(sentences_id=Chap_id)
chap_obj = load_object_from_json(chap_id)
print(chap_obj.content)



pos
neu
neu
neu
neu
neg
neg
pos
neu
neg
[UUID('9759d751-10cd-4322-b253-60a3866a9ee7'), UUID('3de1e89b-d36d-4359-9a1d-b9adad4c54ce')]


In [10]:
# 问题五：在句子中，学生表达了什么样的正向观点
aos1 = Aspect_Sentiment(aspect_term="lunch", opinion="delicious", sentiment="pos")
aos2 = Aspect_Sentiment(aspect_term="dining hall", opinion="frustrating", sentiment="neg")
sentence = Sentence(content=[uuid4()], language="en",aspect_based_sent=[aos1, aos2])

# 解决方案
aos = aspect_based_sentiment_analysis_tool(sentence.id)
print(aos)


[Aspect_Sentiment(aspect_term='lunch', opinion='delicious', sentiment='pos'), Aspect_Sentiment(aspect_term='dining hall', opinion='frustrating', sentiment='neg')]


In [8]:
import uuid
from uuid import UUID, uuid4
from typing import List, Optional
from pydantic import BaseModel, Field, model_validator
import random

# 模拟地图导航的API

class Location(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    name: str
    latitude: float
    longitude: float

class Route(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    start_location: Location
    end_location: Location
    distance: float  # 距离（公里）
    estimated_time: float  # 预计时间（分钟）

class NavigationRequest(BaseModel):
    id: UUID = Field(default_factory=uuid4)
    start_location: Location
    end_location: Location
    mode_of_transport: str  # 交通方式，例如 "driving", "walking", "bicycling"
    
    @model_validator(mode='before')
    @classmethod
    def validate_transport_mode(cls, data: dict) -> dict:
        valid_modes = ['driving', 'walking', 'bicycling']
        if data['mode_of_transport'] not in valid_modes:
            raise ValueError(f"交通方式必须是 {valid_modes}")
        return data

def calculate_distance(start: Location, end: Location) -> float:
    # 计算两点之间的距离（模拟）
    return random.uniform(1.0, 20.0)  # 返回1到20公里之间的随机距离

def calculate_estimated_time(distance: float, mode_of_transport: str) -> float:
    # 根据交通方式和距离计算预计时间（模拟）
    speed = {
        'driving': 60,  # 60公里/小时
        'walking': 5,   # 5公里/小时
        'bicycling': 15  # 15公里/小时
    }
    return distance / speed[mode_of_transport] * 60  # 返回分钟

def create_route(request: NavigationRequest) -> Route:
    distance = calculate_distance(request.start_location, request.end_location)
    estimated_time = calculate_estimated_time(distance, request.mode_of_transport)
    return Route(start_location=request.start_location, end_location=request.end_location, distance=distance, estimated_time=estimated_time)

def get_route_details(route_id: UUID) -> Route:
    # 模拟获取路线详情
    # 实际应用中应从数据库或其他存储中获取
    return Route(id=route_id, start_location=Location(name="起点", latitude=0.0, longitude=0.0),
                 end_location=Location(name="终点", latitude=1.0, longitude=1.0),
                 distance=5.0, estimated_time=10.0)

def navigate(request: NavigationRequest) -> Route:
    # 处理导航请求
    if request.start_location == request.end_location:
        raise ValueError("起点和终点不能相同")
    
    route = create_route(request)
    return route

# 示例用法
location_a = Location(name="地点A", latitude=34.0522, longitude=-118.2437)
location_b = Location(name="地点B", latitude=34.0522, longitude=-118.2437)
request = NavigationRequest(start_location=location_a, end_location=location_b, mode_of_transport="driving")
route = navigate(request)
print(route)

id=UUID('fb38a6db-d932-4d23-9929-c3959ee29b3b') start_location=Location(id=UUID('d1d1e3b8-9b26-4ea8-a2a6-780f7e6bb0b7'), name='地点A', latitude=34.0522, longitude=-118.2437) end_location=Location(id=UUID('e0b3e13e-6b3e-4a50-9b52-63aee7234735'), name='地点B', latitude=34.0522, longitude=-118.2437) distance=2.2757494702922783 estimated_time=2.2757494702922783
